In [1]:
from libs import *

In [2]:
df_vtx = pd.read_excel('data/vertrices.xlsx')
vtx_list = []
# Create the list from the excel filde and stor it as list of objects
for i in range(len(df_vtx)):
    vtx = Vtx(df_vtx['Ver_Name'][i], df_vtx['x'][i], df_vtx['y'][i])
    vtx_list.append(vtx)

In [3]:
vtx_list 

[0(0,0), 1(1,4), 2(2,2), 3(3,3), 4(4,1), 5(5,5)]

In [4]:
obstacles_data = [  
    (1, 0, 2, 0, "T"),  
    (3, 0, 4, 0, "T"),  
    (2, 1, 3, 1, "T"),  
    (1, 1, 1, 2, "N"),  
    (1, 2, 2, 2, "N"),  
    (4, 4, 5, 4, "B"),  
    (5, 4, 5, 5, "B")  
]  

In [5]:
# # Usage example  
# board = Board(obstacles_data) 
# start = vtx_list[0].getName() 
# end = vtx_list[1].getName()

# number_of_paths = vtx_list[0].optimized_paths(vtx_list[1])

# initial_dir = (0,1)
# min_path, min_dis, min_turn_guidance, min_last_direction = board.find_optimized_paths(initial_dir, start, end, number_of_paths)

# print(f'The suitable path is {min_path}')
# print(f'The suitable distance is {min_dis}')
# print(f'The guidance: {min_turn_guidance}')
# print(f'The last direction: {min_last_direction}')

In [6]:
# Usage example  
board = Board(obstacles_data) 
 
start = vtx_list[0].getName() 
end = vtx_list[1].getName()

number_of_paths = vtx_list[0].optimized_paths(vtx_list[1])

all_paths = board.find_paths(start, end, number_of_paths)  

# Count turns for each path  
number = 0   
initial_direction = (1,0)  # Initial direction; can be set based on the context  

for path in all_paths:  
    turns, turn_guidance, last_dir = board.calculate_turns(initial_direction, path)  
    print(f'Path {number + 1}: {path} - Turns needed: {turns}') 
    print(f'The guidance to run: {turn_guidance}')
    print(f'The last direction is {last_dir}') 
    total_time, turns_guidance, last_direction = board.calculate_total_time(initial_direction , path)
    print(f"The total time to traverse the path {path} is: {total_time}")
    print('-'*50)
    number += 1  

print(f'The total number of paths is {number}')  

Path 1: [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4)] - Turns needed: 1
The guidance to run: ['Move Straight', 'Left Turn', 'Move Straight', 'Move Straight', 'Move Straight']
The last direction is (0, 1)
The total time to traverse the path [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4)] is: inf
--------------------------------------------------
Path 2: [(0, 0), (0, 1), (1, 1), (1, 2), (1, 3), (1, 4)] - Turns needed: 3
The guidance to run: ['Left Turn', 'Right Turn', 'Left Turn', 'Move Straight', 'Move Straight']
The last direction is (0, 1)
The total time to traverse the path [(0, 0), (0, 1), (1, 1), (1, 2), (1, 3), (1, 4)] is: inf
--------------------------------------------------
Path 3: [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4)] - Turns needed: 3
The guidance to run: ['Left Turn', 'Move Straight', 'Right Turn', 'Left Turn', 'Move Straight']
The last direction is (0, 1)
The total time to traverse the path [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4)] is: 16
-------------

In [10]:
board = Board(obstacles_data) 
start = vtx_list[5].getName() 
end = vtx_list[4].getName()

number_of_paths = vtx_list[0].optimized_paths(vtx_list[1])

initial_dir = (0,1)
min_path, min_dis, min_turn_guidance, min_last_direction = board.find_optimized_paths(initial_dir, start, end, number_of_paths)

print(f'The suitable path is {min_path}')
print(f'The suitable distance is {min_dis}')
print(f'The guidance: {min_turn_guidance}')
print(f'The last direction: {min_last_direction}')

The suitable path is [(5, 5), (4, 5), (4, 4), (4, 3), (4, 2), (4, 1)]
The suitable distance is 14
The guidance: ['Left Turn', 'Left Turn', 'Move Straight', 'Move Straight', 'Move Straight']
The last direction: (0, -1)


In [21]:
from itertools import permutations  

# Assuming Board and Vertex classes are already defined and imported.  

def all_possible_paths(board, vtx_list, initial_dir):  
    results = []  
    start_vertex = vtx_list[0]  # This is V(0,0)  
    
    # Generate all permutations of the vertices (excluding the start vertex since we're starting from it)  
    vertices_to_permute = vtx_list[1:]  # Starting from the second vertex to create paths  
    all_permutations = permutations(vertices_to_permute)  
    
    for perm in all_permutations:  
        # Create the full path starting and ending at V(0,0)  
        full_path = [start_vertex] + list(perm) + [start_vertex]  # Round trip to start  
        path_names = [v.getName() for v in full_path]  
        
        total_distance = 0  
        full_guidance = []  

        # Now calculate the optimized path for each segment in the full path  
        for i in range(len(full_path) - 1):  
            start_vtx = full_path[i]  
            end_vtx = full_path[i + 1]  
            
            number_of_paths = start_vtx.optimized_paths(end_vtx)  
            min_path, min_dis, min_turn_guidance, min_last_direction = board.find_optimized_paths(initial_dir, start_vtx.getName(), end_vtx.getName(), number_of_paths)  

            # Sum the distances  
            total_distance += min_dis  
            
            # Append guidance in a readable way  
            full_guidance.append(f"From {start_vtx.getName()} to {end_vtx.getName()}: {min_turn_guidance}")  

            # Update the initial direction for the next segment  
            initial_dir = min_last_direction  # Set the last direction as the new initial direction  

        # Store results for this permutation  
        result_entry = {  
            'path': path_names,  
            'total_distance': total_distance,  
            'guidance': full_guidance  
        }  
        results.append(result_entry)  

    return results  

# Assuming obstacles_data and vtx_list is already defined as in your example  
board = Board(obstacles_data)  
initial_dir = (1,0)  

# Call the method to get all possible paths and their details  
all_paths_results = all_possible_paths(board, vtx_list, initial_dir)  

# Print results  
for idx, result in enumerate(all_paths_results):  
    print(f"Path {idx + 1}: {result['path']}")  
    print(f"  Total Distance: {result['total_distance']}")  
    print(f"  Guidance:")  
    for guidance in result['guidance']:  
        print(f"    - {guidance}")  
    print()

# # Print results  
# min_journey = float('inf')
# found_idx = 0
# found_result = []
# for idx, result in enumerate(all_paths_results):  
#     # print(f"Path {idx + 1}: {result['path']}")  
#     # print(f"  Total Distance: {result['total_distance']}")  
#     # print(f"  Guidance:")  
#     # for guidance in result['guidance']:  
#     #     print(f"    - {guidance}")  
#     # print()
#     if result['total_distance'] < min_journey:
#         min_journey = result['total_distance']
#         found_idx = idx + 1
#         found_result = result

Path 1: ['V(0,0)', 'V(1,4)', 'V(2,2)', 'V(3,3)', 'V(4,1)', 'V(5,5)', 'V(0,0)']
  Total Distance: 82
  Guidance:
    - From V(0,0) to V(1,4): ['Left Turn', 'Move Straight', 'Move Straight', 'Move Straight', 'Right Turn']
    - From V(1,4) to V(2,2): ['Move Straight', 'Right Turn', 'Move Straight']
    - From V(2,2) to V(3,3): ['Left Turn', 'Left Turn']
    - From V(3,3) to V(4,1): ['Right Turn', 'Right Turn', 'Move Straight']
    - From V(4,1) to V(5,5): ['Left Turn', 'Left Turn', 'Move Straight', 'Move Straight', 'Move Straight']
    - From V(5,5) to V(0,0): ['Left Turn', 'Move Straight', 'Move Straight', 'Move Straight', 'Move Straight', 'Left Turn', 'Move Straight', 'Move Straight', 'Move Straight', 'Move Straight']

Path 2: ['V(0,0)', 'V(1,4)', 'V(2,2)', 'V(3,3)', 'V(5,5)', 'V(4,1)', 'V(0,0)']
  Total Distance: 76
  Guidance:
    - From V(0,0) to V(1,4): ['U - Turn', 'U - Turn', 'Left Turn', 'Left Turn', 'Move Straight']
    - From V(1,4) to V(2,2): ['Right Turn', 'Right Turn', 'Mov

In [18]:
print(f"Path {found_idx + 1}: {found_result['path']}")  
print(f"  Total Distance: {found_result['total_distance']}")  
print(f"  Guidance:")  
for guidance in found_result['guidance']:  
    print(f"    - {guidance}")  
print()

Path 25: ['V(0,0)', 'V(1,4)', 'V(5,5)', 'V(4,1)', 'V(3,3)', 'V(2,2)', 'V(0,0)']
  Total Distance: 68
  Guidance:
    - From V(0,0) to V(1,4): ['Right Turn', 'Move Straight', 'Move Straight', 'Move Straight', 'Right Turn']
    - From V(1,4) to V(5,5): ['Move Straight', 'Move Straight', 'Move Straight', 'Left Turn', 'Right Turn']
    - From V(5,5) to V(4,1): ['Right Turn', 'Move Straight', 'Move Straight', 'Move Straight', 'Right Turn']
    - From V(4,1) to V(3,3): ['Move Straight', 'Right Turn', 'Move Straight']
    - From V(3,3) to V(2,2): ['Left Turn', 'Left Turn']
    - From V(2,2) to V(0,0): ['Move Straight', 'Move Straight', 'Right Turn', 'Move Straight']

